<a href="https://colab.research.google.com/github/PolyGon-13/DataScience_Study/blob/main/Machine_Learing/08_DBSCAN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# DBSCAN (Density-Based Spatial Clustering of Applications with Noise)

In [113]:
import pandas as pd
import folium

In [115]:
pd.set_option('display.width',None) # 출력화면의 너비
pd.set_option('display.max_rows',100) # 출력할 행의 개수 한도
pd.set_option('display.max_columns',100) # 출력할 열의 개수 한도
pd.set_option('display.max_colwidth',20) # 출력할 열의 너비
pd.set_option('display.unicode.east_asian_width',True) # 유니코드 사용 너비 조정

file_path='./data/middle_school_graduates_report.xlsx'
df=pd.read_excel(file_path)

print(df.columns.values)

['지역' '학교명' '코드' '유형' '주야' '남학생수' '여학생수' '일반고' '특성화고' '과학고' '외고_국제고'
 '예고_체고' '마이스터고' '자사고' '자공고' '기타진학' '취업' '미상' '위도' '경도']


In [116]:
df.head()

,지역,학교명,코드,유형,주야,남학생수,여학생수,일반고,특성화고,과학고,외고_국제고,예고_체고,마이스터고,자사고,자공고,기타진학,취업,미상,위도,경도
0,성북구,서울대학교사범대학부설중학교...,3,국립,주간,277,0,0.585,0.148,0.018,0.007,0.000,0.011,0.227,0.000,0.004,0,0.000,37.594942,127.038909
1,종로구,서울대학교사범대학부설여자중학교...,3,국립,주간,0,256,0.680,0.199,0.000,0.035,0.008,0.000,0.043,0.004,0.031,0,0.000,37.577473,127.003857
2,강남구,개원중학교,3,공립,주간,170,152,0.817,0.047,0.009,0.012,0.003,0.006,0.090,0.003,0.009,0,0.003,37.491637,127.071744
3,강남구,개포중학교,3,공립,주간,83,72,0.755,0.097,0.013,0.013,0.019,0.019,0.065,0.000,0.019,0,0.000,37.480439,127.062201
4,서초구,경원중학교,3,공립,주간,199,212,0.669,0.017,0.007,0.010,0.005,0.000,0.282,0.000,0.010,0,0.000,37.510750,127.008900


In [117]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 415 entries, 0 to 414
Data columns (total 20 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   지역      415 non-null    object 
 1   학교명     415 non-null    object 
 2   코드      415 non-null    int64  
 3   유형      415 non-null    object 
 4   주야      415 non-null    object 
 5   남학생수    415 non-null    int64  
 6   여학생수    415 non-null    int64  
 7   일반고     415 non-null    float64
 8   특성화고    415 non-null    float64
 9   과학고     415 non-null    float64
 10  외고_국제고  415 non-null    float64
 11  예고_체고   415 non-null    float64
 12  마이스터고   415 non-null    float64
 13  자사고     415 non-null    float64
 14  자공고     415 non-null    float64
 15  기타진학    415 non-null    float64
 16  취업      415 non-null    int64  
 17  미상      415 non-null    float64
 18  위도      415 non-null    float64
 19  경도      415 non-null    float64
dtypes: float64(12), int64(4), object(4)
memory usage: 65.0+ KB


In [118]:
df.describe()

,코드,남학생수,여학생수,일반고,특성화고,과학고,외고_국제고,예고_체고,마이스터고,자사고,자공고,기타진학,취업,미상,위도,경도
count,415.000000,415.000000,415.000000,415.000000,415.000000,415.000000,415.000000,415.000000,415.000000,415.000000,415.000000,415.000000,415.0,415.000000,415.000000,415.000000
mean,3.197590,126.532530,116.173494,0.623080,0.149684,0.004378,0.013687,0.017393,0.005251,0.080971,0.034296,0.069571,0.0,0.001670,37.491969,127.032792
std,0.804272,79.217906,76.833082,0.211093,0.102977,0.006739,0.011548,0.092006,0.007557,0.079136,0.080302,0.235630,0.0,0.003697,0.348926,0.265245
min,3.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,34.979940,126.639561
25%,3.000000,80.000000,71.500000,0.566500,0.065500,0.000000,0.006000,0.000000,0.000000,0.027000,0.000000,0.000000,0.0,0.000000,37.501934,126.921758
50%,3.000000,129.000000,118.000000,0.681000,0.149000,0.000000,0.012000,0.005000,0.003000,0.060000,0.000000,0.007000,0.0,0.000000,37.547702,127.013579
75%,3.000000,177.500000,161.500000,0.758000,0.224500,0.006000,0.019000,0.011000,0.007000,0.111500,0.019000,0.015000,0.0,0.003000,37.590670,127.071265
max,9.000000,337.000000,422.000000,0.908000,0.477000,0.055000,0.110000,0.940000,0.046000,0.485000,0.531000,1.000000,0.0,0.036000,37.694777,129.106974


In [120]:
df.isnull().sum().sum()

0

In [121]:
df.duplicated().sum()

0

In [122]:
attr=('© OpenStreetMap contributors')
mschool_map=folium.Map(location=[37.55,126.98],tiles='OpenStreetMap',attr=attr,zoom_start=12)

for name,lat,lng in zip(df['학교명'],df['위도'],df['경도']):
    folium.CircleMarker([lat,lng],
                        radius=5,
                        color='brown',
                        fill=True,
                        fill_color='coral',
                        fill_opacity=0.7,
                        popup=name).add_to(mschool_map)

mschool_map

In [123]:
mschool_map.save('./data/seoul_mschool_location.html')

In [124]:
print(df['지역'].nunique())
print(df['코드'].nunique())
print(df['유형'].nunique())
print(df['주야'].nunique())

25
3
3
1


In [126]:
df_encoded=pd.get_dummies(df,columns=['지역','코드','유형','주야'])
df_encoded.head()

,학교명,남학생수,여학생수,일반고,특성화고,과학고,외고_국제고,예고_체고,마이스터고,자사고,자공고,기타진학,취업,미상,위도,경도,지역_강남구,지역_강동구,지역_강북구,지역_강서구,지역_관악구,지역_광진구,지역_구로구,지역_금천구,지역_노원구,지역_도봉구,지역_동대문구,지역_동작구,지역_마포구,지역_서대문구,지역_서초구,지역_성동구,지역_성북구,지역_송파구,지역_양천구,지역_영등포구,지역_용산구,지역_은평구,지역_종로구,지역_중구,지역_중랑구,코드_3,코드_5,코드_9,유형_공립,유형_국립,유형_사립,주야_주간
0,서울대학교사범대학부설중학교...,277,0,0.585,0.148,0.018,0.007,0.000,0.011,0.227,0.000,0.004,0,0.000,37.594942,127.038909,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,True,False,False,False,True,False,True
1,서울대학교사범대학부설여자중학교...,0,256,0.680,0.199,0.000,0.035,0.008,0.000,0.043,0.004,0.031,0,0.000,37.577473,127.003857,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,True,False,False,False,True,False,True
2,개원중학교,170,152,0.817,0.047,0.009,0.012,0.003,0.006,0.090,0.003,0.009,0,0.003,37.491637,127.071744,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,True,False,False,True
3,개포중학교,83,72,0.755,0.097,0.013,0.013,0.019,0.019,0.065,0.000,0.019,0,0.000,37.480439,127.062201,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,True,False,False,True
4,경원중학교,199,212,0.669,0.017,0.007,0.010,0.005,0.000,0.282,0.000,0.010,0,0.000,37.510750,127.008900,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,True,False,False,True,False,False,True


In [128]:
from sklearn import cluster
from sklearn import preprocessing

train_features=['과학고','외고_국제고','자사고','자공고','유형_공립','유형_국립','유형_사립']
X=df_encoded.loc[:,train_features]

X=preprocessing.StandardScaler().fit_transform(X)

dbm=cluster.DBSCAN(eps=0.2,min_samples=5) # eps : 밀도계산의 기준이 되는 반지름 R, min_samples : 최소 포인트 개수
dbm.fit(X)

cluster_label=dbm.labels_
print(cluster_label) # -1은 노이즈

[-1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1
 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1  4  0
 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1  0 -1 -1 -1  0 -1 -1 -1 -1 -1 -1 -1 -1 -1
 -1 -1  2 -1 -1 -1  0 -1  0 -1 -1 -1  0 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1  0
 -1 -1  0 -1 -1 -1 -1  1  0 -1 -1  0 -1 -1 -1  2 -1 -1 -1 -1 -1 -1  1 -1
 -1 -1 -1  0 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1  0 -1 -1 -1 -1
 -1  2 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1
 -1 -1 -1  0 -1 -1 -1  3 -1 -1 -1 -1 -1 -1 -1  0 -1 -1  0 -1 -1 -1 -1 -1
  2 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1  0 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1  0
 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1
 -1 -1 -1 -1 -1 -1 -1 -1 -1  0 -1 -1 -1 -1  0 -1 -1 -1 -1 -1 -1 -1 -1 -1
 -1 -1 -1 -1 -1 -1  0 -1 -1 -1 -1 -1 -1 -1 -1  4 -1 -1  4 -1 -1 -1 -1 -1
 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1  0 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1
 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1

In [129]:
df_encoded['Cluster']=cluster_label
df_encoded.head()

,학교명,남학생수,여학생수,일반고,특성화고,과학고,외고_국제고,예고_체고,마이스터고,자사고,자공고,기타진학,취업,미상,위도,경도,지역_강남구,지역_강동구,지역_강북구,지역_강서구,지역_관악구,지역_광진구,지역_구로구,지역_금천구,지역_노원구,지역_도봉구,지역_동대문구,지역_동작구,지역_마포구,지역_서대문구,지역_서초구,지역_성동구,지역_성북구,지역_송파구,지역_양천구,지역_영등포구,지역_용산구,지역_은평구,지역_종로구,지역_중구,지역_중랑구,코드_3,코드_5,코드_9,유형_공립,유형_국립,유형_사립,주야_주간,Cluster
0,서울대학교사범대학부설중학교...,277,0,0.585,0.148,0.018,0.007,0.000,0.011,0.227,0.000,0.004,0,0.000,37.594942,127.038909,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,True,False,False,False,True,False,True,-1
1,서울대학교사범대학부설여자중학교...,0,256,0.680,0.199,0.000,0.035,0.008,0.000,0.043,0.004,0.031,0,0.000,37.577473,127.003857,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,True,False,False,False,True,False,True,-1
2,개원중학교,170,152,0.817,0.047,0.009,0.012,0.003,0.006,0.090,0.003,0.009,0,0.003,37.491637,127.071744,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,True,False,False,True,-1
3,개포중학교,83,72,0.755,0.097,0.013,0.013,0.019,0.019,0.065,0.000,0.019,0,0.000,37.480439,127.062201,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,True,False,False,True,-1
4,경원중학교,199,212,0.669,0.017,0.007,0.010,0.005,0.000,0.282,0.000,0.010,0,0.000,37.510750,127.008900,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,True,False,False,True,False,False,True,-1


In [130]:
grouped_cols=['학교명','과학고','외고_국제고','자사고']
grouped=df_encoded.groupby('Cluster')
for key,group in grouped:
    print('* key : ',key)
    print('* number : ',len(group))
    print(group.loc[:,grouped_cols].head())
    print('\n')

* key :  -1
* number :  347
                                학교명  과학고  외고_국제고  자사고
0  서울대학교사범대학부설중학교.....     0.018        0.007   0.227
1  서울대학교사범대학부설여자중학교...   0.000        0.035   0.043
2           개원중학교                   0.009        0.012   0.090
3           개포중학교                   0.013        0.013   0.065
4           경원중학교                   0.007        0.010   0.282


* key :  0
* number :  24
        학교명  과학고  외고_국제고  자사고
47  둔촌중학교     0.0        0.010   0.026
58  성내중학교     0.0        0.013   0.026
62  신명중학교     0.0        0.009   0.031
78  한산중학교     0.0        0.012   0.052
80  강신중학교     0.0        0.012   0.039


* key :  1
* number :  11
             학교명  과학고  외고_국제고  자사고
103      신원중학교     0.0          0.0   0.006
118      개봉중학교     0.0          0.0   0.012
356  서울체육중학교     0.0          0.0   0.000
391    서울광진학교     0.0          0.0   0.000
396    서울정문학교     0.0          0.0   0.000


* key :  2
* number :  5
         학교명  과학고  외고_국제고  자사고
74   천일중학교     0.0        0.003  

In [133]:
colors={-1:'gray',0:'coral',1:'blue',2:'green',3:'red',4:'purple',5:'orange',6:'brown',7:'brick',8:'yellow',9:'magenta',10:'cyan',11:'tan'}

attr=('© OpenStreetMap contributors')
cluster_map=folium.Map(location=[37.55,126.98],tiles='OpenStreetMap',attr=attr,zoom_start=12)

for name,lat,lng,clus in zip(df_encoded['학교명'],df_encoded['위도'],df_encoded['경도'],df_encoded['Cluster']):
    folium.CircleMarker([lat,lng],
                        radius=5,
                        color=colors[clus],
                        fill=True,
                        fill_color=colors[clus],
                        fill_opacity=0.7,
                        popup=name).add_to(cluster_map)
cluster_map